- https://scikit-learn.org/stable/modules/tree.html
- https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html
- https://scikit-learn.org/stable/modules/generated/sklearn.tree.plot_tree.html

In [27]:
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = (20, 6)
plt.rcParams['font.size'] = 14
import pandas as pd

In [28]:
df = pd.read_csv('../data/adult.data', index_col=False)

In [29]:
golden = pd.read_csv('../data/adult.test', index_col=False)

In [14]:
golden.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,salary
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K.
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K.
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K.
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K.
4,18,?,103497,Some-college,10,Never-married,?,Own-child,White,Female,0,0,30,United-States,<=50K.


In [15]:
df.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,salary
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [30]:
df.columns

Index(['age', 'workclass', 'fnlwgt', 'education', 'education-num',
       'marital-status', 'occupation', 'relationship', 'race', 'sex',
       'capital-gain', 'capital-loss', 'hours-per-week', 'native-country',
       'salary'],
      dtype='object')

In [31]:
from sklearn import preprocessing

In [32]:
enc = preprocessing.OrdinalEncoder()

In [33]:
transform_columns = ['sex']
non_num_columns = ['workclass', 'education', 'marital-status', 
                     'occupation', 'relationship', 'race', 'sex', 
                     'native-country']

In [34]:
pd.get_dummies(df[transform_columns]).head()

,sex_ Female,sex_ Male
0,0,1
1,0,1
2,0,1
3,0,1
4,1,0


In [36]:
x = df.copy()

x = pd.concat([x.drop(non_num_columns, axis=1), 
               pd.get_dummies(df[transform_columns])], axis=1,)

x["salary"] = enc.fit_transform(df[["salary"]])

In [22]:
x.head()

,age,fnlwgt,education-num,capital-gain,capital-loss,hours-per-week,salary,sex_ Female,sex_ Male
0,39,77516,13,2174,0,40,0.0,0,1
1,50,83311,13,0,0,13,0.0,0,1
2,38,215646,9,0,0,40,0.0,0,1
3,53,234721,7,0,0,40,0.0,0,1
4,28,338409,13,0,0,40,0.0,1,0


In [37]:
xt = golden.copy()

xt = pd.concat([xt.drop(non_num_columns, axis=1), 
               pd.get_dummies(golden[transform_columns])], axis=1,)

xt["salary"] = enc.fit_transform(golden[["salary"]])

In [24]:
xt.salary.value_counts()

0.0    12435
1.0     3846
Name: salary, dtype: int64

In [25]:
enc.categories_

[array([' <=50K.', ' >50K.'], dtype=object)]

In [38]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier

#### Choose the model of your preference: DecisionTree or RandomForest

In [51]:
model = RandomForestClassifier(criterion='entropy')

In [39]:
model = DecisionTreeClassifier(criterion='entropy', max_depth=None)

In [52]:
model.fit(x.drop(['fnlwgt','salary'], axis=1), x.salary)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='entropy', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [53]:
model.tree_.node_count

AttributeError: 'RandomForestClassifier' object has no attribute 'tree_'

In [54]:
list(zip(x.drop(['fnlwgt','salary'], axis=1).columns, model.feature_importances_))

[('age', 0.34257663939594074),
 ('education-num', 0.17141963893382645),
 ('capital-gain', 0.1838485744532467),
 ('capital-loss', 0.06966120922295958),
 ('hours-per-week', 0.1752692539095822),
 ('sex_ Female', 0.029561014503859014),
 ('sex_ Male', 0.02766366958058538)]

In [43]:
list(zip(x.drop(['fnlwgt','salary'], axis=1).columns, model.feature_importances_))

[('age', 0.3225996240681743),
 ('education-num', 0.16178710998511167),
 ('capital-gain', 0.22735010841757017),
 ('capital-loss', 0.07846770837020521),
 ('hours-per-week', 0.1541319966167697),
 ('sex_ Female', 0.03418477843918968),
 ('sex_ Male', 0.021478674102979156)]

In [22]:
x.drop(['fnlwgt','salary'], axis=1).head()

,age,education-num,capital-gain,capital-loss,hours-per-week,sex_ Female,sex_ Male
0,39,13,2174,0,40,0,1
1,50,13,0,0,13,0,1
2,38,9,0,0,40,0,1
3,53,7,0,0,40,0,1
4,28,13,0,0,40,1,0


In [23]:
set(x.columns) - set(xt.columns)

set()

In [24]:
list(x.drop('salary', axis=1).columns)

['age',
 'fnlwgt',
 'education-num',
 'capital-gain',
 'capital-loss',
 'hours-per-week',
 'sex_ Female',
 'sex_ Male']

In [55]:
predictions = model.predict(xt.drop(['fnlwgt','salary'], axis=1))
predictionsx = model.predict(x.drop(['fnlwgt','salary'], axis=1))

In [39]:
from sklearn.metrics import (
    accuracy_score, 
    classification_report, 
    confusion_matrix, auc, roc_curve
)

In [45]:
accuracy_score(xt.salary, predictions)

0.8213868926970088

In [57]:
accuracy_score(xt.salary, predictions)

0.82808181315644

In [46]:
confusion_matrix(xt.salary, predictions)

array([[11472,   963],
       [ 1945,  1901]])

In [47]:
print(classification_report(xt.salary, predictions))

              precision    recall  f1-score   support

         0.0       0.86      0.92      0.89     12435
         1.0       0.66      0.49      0.57      3846

    accuracy                           0.82     16281
   macro avg       0.76      0.71      0.73     16281
weighted avg       0.81      0.82      0.81     16281



In [56]:
print(classification_report(xt.salary, predictions))

              precision    recall  f1-score   support

         0.0       0.86      0.92      0.89     12435
         1.0       0.68      0.52      0.59      3846

    accuracy                           0.83     16281
   macro avg       0.77      0.72      0.74     16281
weighted avg       0.82      0.83      0.82     16281



In [48]:
accuracy_score(x.salary, predictionsx)

0.8955806025613464

In [49]:
confusion_matrix(x.salary, predictionsx)

array([[24097,   623],
       [ 2777,  5064]])

In [50]:
print(classification_report(x.salary, predictionsx))

              precision    recall  f1-score   support

         0.0       0.90      0.97      0.93     24720
         1.0       0.89      0.65      0.75      7841

    accuracy                           0.90     32561
   macro avg       0.89      0.81      0.84     32561
weighted avg       0.90      0.90      0.89     32561



In [58]:
print(classification_report(x.salary, predictionsx))

              precision    recall  f1-score   support

         0.0       0.91      0.96      0.93     24720
         1.0       0.85      0.68      0.76      7841

    accuracy                           0.90     32561
   macro avg       0.88      0.82      0.85     32561
weighted avg       0.89      0.90      0.89     32561



# For the following use the above `adult` dataset. Start with only numerical features/columns.  

In [104]:
df = pd.read_csv('../data/adult.data', index_col=False)
golden = pd.read_csv('../data/adult.test', index_col=False)
df["marital-status"].replace(" Never-Married", 'Never-Married')
df["native-country"].replace(' United-States', 'United-States')
enc = preprocessing.OrdinalEncoder()
transform_columns = ['sex']
non_num_columns = ['workclass', 'education', 'marital-status', 
                     'occupation', 'relationship', 'race', 'sex', 
                     'native-country']
x = df.copy()
x = pd.concat([x.drop(non_num_columns, axis=1), 
               pd.get_dummies(df[transform_columns])], axis=1,)
x["salary"] = enc.fit_transform(df[["salary"]])
xt = golden.copy()
xt = pd.concat([xt.drop(non_num_columns, axis=1), 
               pd.get_dummies(golden[transform_columns])], axis=1,)
xt["salary"] = enc.fit_transform(golden[["salary"]])

,age,fnlwgt,education-num,capital-gain,capital-loss,hours-per-week,salary
0,39,77516,13,2174,0,40,0.0
1,50,83311,13,0,0,13,0.0
2,38,215646,9,0,0,40,0.0
3,53,234721,7,0,0,40,0.0
4,28,338409,13,0,0,40,0.0
...,...,...,...,...,...,...,...
32556,27,257302,12,0,0,38,0.0
32557,40,154374,9,0,0,40,1.0
32558,58,151910,9,0,0,40,0.0
32559,22,201490,9,0,0,20,0.0


# 1. Show the RandomForest outperforms the DecisionTree for a fixed `max_depth` by training using the train set and `precision`, `recall`, `f1` on golden-test set.

In [54]:
RF_model = RandomForestClassifier(criterion='entropy', max_depth = 8)
DT_model = DecisionTreeClassifier(criterion='entropy', max_depth= 8)
RF_model.fit(x.drop(['fnlwgt','salary'], axis=1), x.salary)
DT_model.fit(x.drop(['fnlwgt','salary'], axis=1), x.salary)
RF_predictions = RF_model.predict(xt.drop(['fnlwgt','salary'], axis=1))
DT_predictions = DT_model.predict(xt.drop(['fnlwgt','salary'], axis=1))
RF_predictionsx = RF_model.predict(x.drop(['fnlwgt','salary'], axis=1))
DT_predictionsx = DT_model.predict(x.drop(['fnlwgt','salary'], axis=1))

# show RandomForest (RF) outperforms the DecisionTree (DT)
RF_accuracy = accuracy_score(x.salary, RF_predictionsx)
DT_accuracy = accuracy_score(x.salary, DT_predictionsx)
RF_confusion = confusion_matrix(x.salary, RF_predictionsx)
DT_confusion = confusion_matrix(x.salary, DT_predictionsx)
RF_accuracy, DT_accuracy, RF_confusion, DT_confusion

(0.8453671570283467,
 0.8405454377936795,
 array([[23892,   828],
        [ 4207,  3634]], dtype=int64),
 array([[23352,  1368],
        [ 3824,  4017]], dtype=int64))

In [55]:
# classification report showing precision, recall and f1 on golden-test set (xt)
RF_class = classification_report(xt.salary, RF_predictions)
DT_class = classification_report(xt.salary, DT_predictions)
print(RF_class, DT_class)

              precision    recall  f1-score   support

         0.0       0.85      0.96      0.90     12435
         1.0       0.78      0.44      0.56      3846

    accuracy                           0.84     16281
   macro avg       0.81      0.70      0.73     16281
weighted avg       0.83      0.84      0.82     16281
               precision    recall  f1-score   support

         0.0       0.86      0.94      0.90     12435
         1.0       0.72      0.49      0.58      3846

    accuracy                           0.84     16281
   macro avg       0.79      0.72      0.74     16281
weighted avg       0.83      0.84      0.82     16281



# 2. For RandomForest or DecisionTree and using the `adult` dataset, systematically add new columns, one by one, that are non-numerical but converted using the feature-extraction techniques we learned. Show [`precision`, `recall`, `f1`] for each additional feature added.

In [107]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
x = df.copy()
non_num_columns = ['salary', 'education', 'marital-status', 
                     'occupation', 'relationship', 'race', 'sex', 
                     'native-country']
x = pd.concat([x.drop(non_num_columns, axis=1)], axis=1,)
RF_model = RandomForestClassifier(criterion='entropy')
x["workclass"] = label_encoder.fit_transform(df[["workclass"]])
RF_model.fit(x.drop(['fnlwgt','workclass'], axis=1), x.workclass)
RF_predictions_work = RF_model.predict(x.drop(['fnlwgt','workclass'], axis=1))
RF_class_work = classification_report(x.workclass, RF_predictions_work)
x = df.copy()
non_num_columns = ['salary', 'workclass', 'marital-status', 
                     'occupation', 'relationship', 'race', 'sex', 
                     'native-country']
x = pd.concat([x.drop(non_num_columns, axis=1)], axis=1,)
x["education"] = label_encoder.fit_transform(df[["education"]])
RF_model.fit(x.drop(['fnlwgt','education'], axis=1), x.education)
RF_predictions_ed = RF_model.predict(x.drop(['fnlwgt','education'], axis=1))
RF_class_ed = classification_report(x.education, RF_predictions_ed)
x = df.copy()
non_num_columns = ['salary', 'education', 'marital-status', 
                     'workclass', 'relationship', 'race', 'sex', 
                     'native-country']
x = pd.concat([x.drop(non_num_columns, axis=1)], axis=1,)
x["occupation"] = label_encoder.fit_transform(df[["occupation"]])
RF_model.fit(x.drop(['fnlwgt','occupation'], axis=1), x.occupation)
RF_predictions_occ = RF_model.predict(x.drop(['fnlwgt','occupation'], axis=1))
RF_class_occ = classification_report(x.occupation, RF_predictions_occ)
x = df.copy()
non_num_columns = ['salary', 'education', 'marital-status', 
                     'occupation', 'workclass', 'race', 'sex', 
                     'native-country']
x = pd.concat([x.drop(non_num_columns, axis=1)], axis=1,)
x["relationship"] = label_encoder.fit_transform(df[["relationship"]])
RF_model.fit(x.drop(['fnlwgt','relationship'], axis=1), x.relationship)
RF_predictions_relation = RF_model.predict(x.drop(['fnlwgt','relationship'], axis=1))
RF_class_relation = classification_report(x.relationship, RF_predictions_relation)
x = df.copy()
non_num_columns = ['salary', 'education', 'marital-status', 
                     'occupation', 'relationship', 'workclass', 'sex', 
                     'native-country']
x = pd.concat([x.drop(non_num_columns, axis=1)], axis=1,)
x["race"] = label_encoder.fit_transform(df[["race"]])
RF_model.fit(x.drop(['fnlwgt','race'], axis=1), x.race)
RF_predictions_race = RF_model.predict(x.drop(['fnlwgt','race'], axis=1))
RF_class_race = classification_report(x.race, RF_predictions_race)
x = df.copy()
non_num_columns = ['salary', 'education', 'workclass', 
                     'occupation', 'relationship', 'race', 'sex', 
                     'native-country']
x = pd.concat([x.drop(non_num_columns, axis=1)], axis=1,)
x["marital-status"] = label_encoder.fit_transform(df[["marital-status"]])
RF_model.fit(x.drop(['fnlwgt','marital-status'], axis=1), x["marital-status"])
RF_predictions_mar = RF_model.predict(x.drop(['fnlwgt','marital-status'], axis=1))
RF_class_mar = classification_report(x["marital-status"], RF_predictions_mar)
x = df.copy()
non_num_columns = ['salary', 'education', 'marital-status', 
                     'occupation', 'relationship', 'race', 'sex', 
                     'workclass']
x = pd.concat([x.drop(non_num_columns, axis=1)], axis=1,)
x["native-country"] = label_encoder.fit_transform(df[["native-country"]])
RF_model.fit(x.drop(['fnlwgt','native-country'], axis=1), x["native-country"])
RF_predictions_nat = RF_model.predict(x.drop(['fnlwgt','native-country'], axis=1))
RF_class_nat = classification_report(x["native-country"], RF_predictions_nat)
print(RF_class_work, RF_class_ed,RF_class_occ,RF_class_relation,RF_class_race, RF_class_mar, RF_class_nat)

C:\Users\walte\anaconda3\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
C:\Users\walte\anaconda3\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
C:\Users\walte\anaconda3\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
C:\Users\walte\anaconda3\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**

              precision    recall  f1-score   support

           0       0.86      0.42      0.57      1836
           1       0.86      0.23      0.36       960
           2       0.85      0.34      0.48      2093
           3       0.50      0.14      0.22         7
           4       0.80      0.98      0.88     22696
           5       0.84      0.45      0.59      1116
           6       0.82      0.47      0.60      2541
           7       0.82      0.32      0.46      1298
           8       1.00      0.50      0.67        14

    accuracy                           0.80     32561
   macro avg       0.82      0.43      0.54     32561
weighted avg       0.81      0.80      0.77     32561
               precision    recall  f1-score   support

           0       1.00      1.00      1.00       933
           1       1.00      1.00      1.00      1175
           2       1.00      1.00      1.00       433
           3       1.00      1.00      1.00       168
           4       1.00 

# 3. Optional: Using gridSearch find the most optimal parameters for your model
Warning: this can be computationally intensive and may take some time.
- https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html
- https://scikit-learn.org/stable/modules/grid_search.html